In [6]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [8]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [9]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [10]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary

uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [11]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary

    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [12]:
#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [05:29<00:00,  1.65s/it]
[MainProcess/INFO] experiments finished


In [13]:
#observing the simulation runs
experiments, outcomes = results

In [14]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{'0_RfR 0':1,
                                  '0_RfR 1':1,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':5}),
           Policy('policy 2', **{'4_RfR 0':1,
                                  '4_RfR 1':1,
                                  '4_RfR 2':1,
                                  'A.5_DikeIncrease 0':5}),
           Policy('policy 3', **{'1_RfR 0':1,
                                  '2_RfR 1':1,
                                  '3_RfR 2':1,
                                  'A.3_DikeIncrease 0':5})]

In [15]:
#pass the policies list to EMA workbench experiment runs
n_scenarios = 50
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 50 scenarios * 3 policies * 1 model(s) = 150 experiments
100%|████████████████████████████████████████| 150/150 [01:01<00:00,  2.42it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [26]:
# from ema_workbench import Constraint
#
# constraints = [
#     Constraint("a1 total cost", outcome_names="A.1 Total Costs", function=lambda x: max(0, x - 1))
# ]

In [34]:
from ema_workbench.em_framework.optimization import HyperVolume, EpsilonProgress

convergence_metrics = [
    HyperVolume(minimum=[0,0,0], maximum=[1.1, 1.1, 1.1]), #from \"Optimization Moro Test.ipynb\"
    EpsilonProgress(),
]

In [35]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(
        nfe=50,
        searchover="levers",
        epsilons=[
                     0.1,
                 ]
                 * len(dike_model.outcomes),
        #constraints=constraints
        convergence=convergence_metrics
)



  0%|                                                   | 0/50 [00:00<?, ?it/s]

100it [00:36,  2.73it/s]                                                       


IndexError: list index out of range

In [36]:
results

,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,3_RfR 0,...,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,0,0,0,0,1,1,1,1,0,0,...,2.764751e+08,0.014444,1.137879e+08,0.0,4.721000e+07,0.000000,1.198253e+08,0.002501,8.743000e+08,11017.691395
1,0,1,0,1,1,1,1,0,0,1,...,2.305818e+08,0.000095,8.920564e+07,0.0,1.936463e+07,0.000000,1.684455e+08,0.000000,1.267200e+09,27.971177
2,1,1,1,0,0,1,1,0,1,1,...,2.390190e+08,0.000000,7.883842e+07,0.0,4.614800e+07,0.000000,1.857729e+08,0.003221,1.543700e+09,1247.780435
3,0,1,1,1,1,1,1,0,1,1,...,4.100723e+08,0.069002,6.911389e+07,0.0,2.673734e+07,0.000000,1.374624e+08,0.000000,1.126400e+09,6447.917667
4,1,0,0,1,0,0,0,0,0,0,...,5.217284e+08,0.024252,5.040648e+07,0.0,4.242224e+07,0.000000,1.243025e+08,0.000751,4.236000e+08,16189.847829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0,0,0,0,1,0,1,1,0,0,...,8.266463e+07,0.000151,7.068085e+07,0.0,2.124459e+07,0.000015,2.015526e+08,0.000000,2.792000e+08,6961.446843
72,1,1,1,1,0,0,1,1,0,0,...,4.997543e+08,0.021302,1.087102e+08,0.0,1.917024e+07,0.000000,1.463574e+08,0.000000,1.543700e+09,14543.459712
73,0,1,1,1,0,0,0,0,0,0,...,2.126218e+08,0.012231,8.559858e+07,0.0,4.797029e+07,0.000000,1.359580e+08,0.000000,5.082000e+08,959.995676
74,1,0,0,1,1,1,1,1,1,0,...,2.660916e+08,0.006087,6.226744e+07,0.0,2.437901e+07,0.000000,4.159127e+08,0.030883,1.086200e+09,22455.081420


In [33]:
# change outcomes so direction is undesirable
minimize = ScalarOutcome.MINIMIZE
maximize = ScalarOutcome.MAXIMIZE

for outcome in dike_model.outcomes:
    if outcome.kind == minimize:
        outcome.kind = maximize
    else:
        outcome.kind = minimize

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(
        nfe=1000,
        searchover="uncertainties",
        epsilons=[
                     0.1,
                 ]
                 * len(dike_model.outcomes),
    )

[MainProcess/INFO] pool started with 12 workers

  0%|                                                 | 0/1000 [00:00<?, ?it/s]Traceback (most recent call last):
  File "C:\Users\python\anaconda3\lib\site-packages\ema_workbench\em_framework\callbacks.py", line 246, in _store_outcomes
    self.results[outcome][case_id,] = outcome_res
KeyError: 'RfR Total Costs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\python\anaconda3\lib\site-packages\ema_workbench\em_framework\ema_multiprocessing.py", line 253, in run
    self.callback(*result.get())
  File "C:\Users\python\anaconda3\lib\site-packages\ema_workbench\em_framework\callbacks.py", line 281, in __call__
    self._store_outcomes(experiment.experiment_id, outcomes)
  File "C:\Users\python\anaconda3\lib\site-packages\ema_workbench\em_framework\callbacks.py", line 260, in _store_outcomes
    self.results[outcome][:] = np.nan
ValueError: cannot convert float NaN to

ValueError: cannot convert float NaN to integer

In [ ]:
results